# Attempt at Pytorch use

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# Import the utility functions
from animal_classification import get_pytorch_datasets, display_translated_labels

# Set the directory path for images
directory = r'csv/raw-img'

# Get PyTorch datasets and dataloaders
train_dataset, val_dataset = get_pytorch_datasets(directory)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Display and translate class labels
class_labels, translated_labels = display_translated_labels(train_dataset)


ImportError: cannot import name 'get_pytorch_datasets' from 'animal_classification' (c:\Users\kodom\CIS377\CIS377-Animal-Classification\CIS377-Animal-Classification\animal_classification.py)

### Load the Model (pre-trained)

In [ ]:
# Load the pre-trained ResNet model
base_model = models.resnet50(pretrained=True)
for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers

In [ ]:
# Unfreeze the last 10 layers
for param in list(base_model.parameters())[-10:]:
    param.requires_grad = True

# Modify the final layers
num_classes = len(translated_labels)  # Assuming 10 classes
base_model.fc = nn.Sequential(
    nn.Linear(base_model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)
)

In [ ]:
# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = base_model.to(device)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(base_model.parameters(), lr=0.001)

In [ ]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = correct / total
        train_loss /= len(train_loader)

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_acc = correct / total
        val_loss /= len(val_loader)

        # Store metrics
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
              f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    return history

# Train the model
history = train_model(base_model, train_loader, val_loader, criterion, optimizer, epochs=5)

In [ ]:
# Plot the training history
def plot_pytorch_training_history(history, metrics):
    valid_metrics = [metric for metric in metrics if metric in history]
    for metric in valid_metrics:
        plt.plot(history[metric], label=metric)
    plt.legend()
    plt.show()

metrics = ['train_loss', 'val_loss', 'train_acc', 'val_acc']
plot_pytorch_training_history(history, metrics)

In [ ]:
# Evaluation and confusion matrix
base_model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = base_model(images)
        _, predicted = outputs.max(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=translated_labels))

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=translated_labels, yticklabels=translated_labels)
plt.show()